In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd

from src.configurations import Configuration

config = Configuration()
df = pd.read_csv(config.final_filtered_csv)
df['datetime'] = pd.to_datetime(df['datetime'])
df = df.set_index(['id', 'datetime'])
df.info()

2025-06-30 09:08:49.703 | INFO     | src.config:<module>:11 - PROJ_ROOT path is: C:\Users\ross\OneDrive\Documents\Masters\Project\masters_project


<class 'pandas.core.frame.DataFrame'>
MultiIndex: 8640 entries, (np.int64(41131654), Timestamp('2019-09-26 17:00:00')) to (np.int64(86025410), Timestamp('2017-10-08 10:30:00'))
Data columns (total 19 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  8640 non-null   int64  
 1   iob mean    8640 non-null   float64
 2   cob mean    8629 non-null   float64
 3   bg mean     8640 non-null   float64
 4   iob min     8640 non-null   float64
 5   cob min     8629 non-null   float64
 6   bg min      8640 non-null   float64
 7   iob max     8640 non-null   float64
 8   cob max     8629 non-null   float64
 9   bg max      8640 non-null   float64
 10  iob std     7818 non-null   float64
 11  cob std     7800 non-null   float64
 12  bg std      7818 non-null   float64
 13  iob count   8640 non-null   int64  
 14  cob count   8640 non-null   int64  
 15  bg count    8640 non-null   int64  
 16  offset      8640 non-null   int64  
 17  day   

C:\Users\ross\AppData\Local\Temp\ipykernel_48808\1242868785.py:10: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['datetime'] = pd.to_datetime(df['datetime'])


In [2]:
# Add the weekday/weekend classification to the df
df['day_type'] = df.index.get_level_values('datetime').weekday.map(lambda x: 'weekend' if x >= 5 else 'weekday').astype('category')
df = pd.get_dummies(df, columns=['day_type'], prefix='day_type')
df.head()

Unnamed: 0  iob mean  cob mean  bg mean  \
id       datetime                                                       
41131654 2019-09-26 17:00:00           0     1.725      0.00  152.500   
         2019-09-26 17:30:00           1     0.549      0.00  134.667   
         2019-09-26 18:00:00           2     2.594     35.75  192.250   
         2019-09-26 18:30:00           3     3.251      9.60  224.000   
         2019-09-26 19:00:00           4     2.251      1.00  206.000   

                              iob min  cob min  bg min  iob max  cob max  \
id       datetime                                                          
41131654 2019-09-26 17:00:00    1.500      0.0   146.0    1.928      0.0   
         2019-09-26 17:30:00    0.263      0.0   129.0    1.069      0.0   
         2019-09-26 18:00:00    1.525     30.0   169.0    3.243     40.0   
         2019-09-26 18:30:00    3.030      0.0   220.0    3.434     19.0   
         2019-09-26 19:00:00    1.692      0.0   202.0    2.595      3.0   

                              bg max  ...  cob std  bg std  iob count  \
id       datetime                     ...                               
41131654 2019-09-26 17:00:00   158.0  ...    0.000   5.916          4   
         2019-09-26 17:30:00   138.0  ...    0.000   4.933          3   
         2019-09-26 18:00:00   206.0  ...    5.058  17.347          4   
         2019-09-26 18:30:00   228.0  ...    9.017   3.536          5   
         2019-09-26 19:00:00   209.0  ...    1.732   3.606          3   

                              cob count  bg count  offset         day  \
id       datetime                                                       
41131654 2019-09-26 17:00:00          4         4      12  26/09/2019   
         2019-09-26 17:30:00          3         3      12  26/09/2019   
         2019-09-26 18:00:00          4         4      12  26/09/2019   
         2019-09-26 18:30:00          5         5      12  26/09/2019   
         2019-09-26 19:00:00          3         3      12  26/09/2019   

                                  time day_type_weekday  day_type_weekend  
id       datetime                                                          
41131654 2019-09-26 17:00:00  17:00:00             True             False  
         2019-09-26 17:30:00  17:30:00             True             False  
         2019-09-26 18:00:00  18:00:00             True             False  
         2019-09-26 18:30:00  18:30:00             True             False  
         2019-09-26 19:00:00  19:00:00             True             False  

[5 rows x 21 columns]

In [3]:
# Add rate of change columns based on iob, cob, bg mean columns
# First add a column for the interval between the previous value and the next. The first value for an id will be NaN
import numpy as np
df['time_diff'] = df.index.get_level_values('datetime').diff()
first_idx = ~df.index.get_level_values('id').duplicated()
df.loc[first_idx, 'time_diff'] = np.nan
df.head()

interval = pd.Timedelta('30min')
# Then add rate columns
for col in ['iob mean', 'cob mean', 'bg mean']:
    value_diff = df[col].groupby(df.index.get_level_values('id')).diff()
    rate_of_change = value_diff.where(df['time_diff'] == interval)
    df[f'{col} rate_of_change'] = rate_of_change

df.head()

Unnamed: 0  iob mean  cob mean  bg mean  \
id       datetime                                                       
41131654 2019-09-26 17:00:00           0     1.725      0.00  152.500   
         2019-09-26 17:30:00           1     0.549      0.00  134.667   
         2019-09-26 18:00:00           2     2.594     35.75  192.250   
         2019-09-26 18:30:00           3     3.251      9.60  224.000   
         2019-09-26 19:00:00           4     2.251      1.00  206.000   

                              iob min  cob min  bg min  iob max  cob max  \
id       datetime                                                          
41131654 2019-09-26 17:00:00    1.500      0.0   146.0    1.928      0.0   
         2019-09-26 17:30:00    0.263      0.0   129.0    1.069      0.0   
         2019-09-26 18:00:00    1.525     30.0   169.0    3.243     40.0   
         2019-09-26 18:30:00    3.030      0.0   220.0    3.434     19.0   
         2019-09-26 19:00:00    1.692      0.0   202.0    2.595      3.0   

                              bg max  ...  bg count  offset         day  \
id       datetime                     ...                                 
41131654 2019-09-26 17:00:00   158.0  ...         4      12  26/09/2019   
         2019-09-26 17:30:00   138.0  ...         3      12  26/09/2019   
         2019-09-26 18:00:00   206.0  ...         4      12  26/09/2019   
         2019-09-26 18:30:00   228.0  ...         5      12  26/09/2019   
         2019-09-26 19:00:00   209.0  ...         3      12  26/09/2019   

                                  time  day_type_weekday  day_type_weekend  \
id       datetime                                                            
41131654 2019-09-26 17:00:00  17:00:00              True             False   
         2019-09-26 17:30:00  17:30:00              True             False   
         2019-09-26 18:00:00  18:00:00              True             False   
         2019-09-26 18:30:00  18:30:00              True             False   
         2019-09-26 19:00:00  19:00:00              True             False   

                                   time_diff iob mean rate_of_change  \
id       datetime                                                      
41131654 2019-09-26 17:00:00             NaT                     NaN   
         2019-09-26 17:30:00 0 days 00:30:00                  -1.176   
         2019-09-26 18:00:00 0 days 00:30:00                   2.045   
         2019-09-26 18:30:00 0 days 00:30:00                   0.657   
         2019-09-26 19:00:00 0 days 00:30:00                  -1.000   

                             cob mean rate_of_change  bg mean rate_of_change  
id       datetime                                                             
41131654 2019-09-26 17:00:00                     NaN                     NaN  
         2019-09-26 17:30:00                    0.00                 -17.833  
         2019-09-26 18:00:00                   35.75                  57.583  
         2019-09-26 18:30:00                  -26.15                  31.750  
         2019-09-26 19:00:00                   -8.60                 -18.000  

[5 rows x 25 columns]

Now we have some idea of the sorts of features, we can look at scripting the functions in our class to build the feature set. We break down the features derived from the random variables and time series:
1. Time-based variables:
a) One-hot variable identifying weekday/weekends
b) The hour of the day of the time series
c) Trigonometric functions of the timestamps
2. Resampled means
a) Rate of change from previous interval
b) Rate of change from previous 2 hours
c) Hourly mean
3. Resampled maximums
a) Peaks greater than the mean for the time series
The mean won't necessarily be the best value here.

In [4]:
from src.features import FeatureSet

features = FeatureSet(input_path=config.final_filtered_csv, sample_rate=30)

scale_columns = ['iob mean', 'cob mean', 'bg mean',
                 'iob min', 'cob min', 'bg min',
                 'iob max', 'cob max', 'bg max',
                 'iob std', 'cob std', 'bg std']

# Add all features
features.add_time_based_features()
features.add_day_type()
features.add_rate_of_change()
features.add_peaks_above_mean()
features.add_hourly_mean()

# Scale variables
features.scale_features()

Scaling ['iob mean', 'cob mean', 'bg mean', 'iob min', 'cob min', 'bg min', 'iob max', 'cob max', 'bg max', 'hour_of_day', 'hour_sin', 'hour_cos', 'day_type_weekday', 'day_type_weekend', 'iob mean_rate_of_change', 'cob mean_rate_of_change', 'bg mean_rate_of_change', 'iob max_peaks_above_mean', 'cob max_peaks_above_mean', 'bg max_peaks_above_mean', 'iob mean hourly_mean', 'cob mean hourly_mean', 'bg mean hourly_mean'] columns


C:\Users\ross\OneDrive\Documents\Masters\Project\masters_project\src\helper.py:77: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['datetime'] = pd.to_datetime(df['datetime'])


In [6]:
features.df[features.all_feature_cols].head()

Empty DataFrame
Columns: []
Index: [(41131654, 2019-09-26 17:00:00), (41131654, 2019-09-26 17:30:00), (41131654, 2019-09-26 18:00:00), (41131654, 2019-09-26 18:30:00), (41131654, 2019-09-26 19:00:00)]

In [22]:
features = FeatureSet(input_path=config.final_filtered_csv, sample_rate=30)

df = features.get_all_features()

Scaling ['iob mean', 'cob mean', 'bg mean', 'iob min', 'cob min', 'bg min', 'iob max', 'cob max', 'bg max', 'day_type_weekday', 'day_type_weekend', 'iob mean_rate_of_change', 'cob mean_rate_of_change', 'bg mean_rate_of_change', 'iob mean hourly_mean', 'cob mean hourly_mean', 'bg mean hourly_mean', 'iob max_peaks_above_mean', 'cob max_peaks_above_mean', 'bg max_peaks_above_mean', 'hour_of_day', 'hour_sin', 'hour_cos'] columns


In [23]:
df.head()

iob mean  cob mean   bg mean   iob min  \
id       datetime                                                      
41131654 2019-09-12 18:00:00  0.270057  0.000000  0.623390  0.243616   
         2019-09-12 18:30:00  0.285132  0.325000  0.602210  0.231076   
         2019-09-12 19:00:00  0.372554  0.295833  0.617403  0.359405   
         2019-09-12 19:30:00  0.260481  0.025000  0.593923  0.284943   
         2019-09-12 21:30:00  0.084215  0.000000  0.261740  0.092494   

                               cob min    bg min   iob max   cob max  \
id       datetime                                                      
41131654 2019-09-12 18:00:00  0.000000  0.602210  0.293407  0.000000   
         2019-09-12 18:30:00  0.308333  0.591160  0.330379  0.330579   
         2019-09-12 19:00:00  0.283333  0.613260  0.400609  0.305785   
         2019-09-12 19:30:00  0.025000  0.593923  0.254465  0.024793   
         2019-09-12 21:30:00  0.000000  0.240331  0.120272  0.000000   

                                bg max  day_type_weekday  ...  \
id       datetime                                         ...   
41131654 2019-09-12 18:00:00  0.649171               1.0  ...   
         2019-09-12 18:30:00  0.613260               1.0  ...   
         2019-09-12 19:00:00  0.621547               1.0  ...   
         2019-09-12 19:30:00  0.593923               1.0  ...   
         2019-09-12 21:30:00  0.303867               1.0  ...   

                              bg mean_rate_of_change  iob mean hourly_mean  \
id       datetime                                                            
41131654 2019-09-12 18:00:00                     NaN              0.154347   
         2019-09-12 18:30:00                0.406787              0.154347   
         2019-09-12 19:00:00                0.446809              0.202337   
         2019-09-12 19:30:00                0.404255              0.202337   
         2019-09-12 21:30:00                     NaN              0.089872   

                              cob mean hourly_mean  bg mean hourly_mean  \
id       datetime                                                         
41131654 2019-09-12 18:00:00              0.238111             0.494787   
         2019-09-12 18:30:00              0.238111             0.494787   
         2019-09-12 19:00:00              0.175808             0.648843   
         2019-09-12 19:30:00              0.175808             0.648843   
         2019-09-12 21:30:00              0.060551             0.450676   

                              iob max_peaks_above_mean  \
id       datetime                                        
41131654 2019-09-12 18:00:00                       0.0   
         2019-09-12 18:30:00                       0.0   
         2019-09-12 19:00:00                       1.0   
         2019-09-12 19:30:00                       0.0   
         2019-09-12 21:30:00                       0.0   

                              cob max_peaks_above_mean  \
id       datetime                                        
41131654 2019-09-12 18:00:00                       0.0   
         2019-09-12 18:30:00                       1.0   
         2019-09-12 19:00:00                       0.0   
         2019-09-12 19:30:00                       0.0   
         2019-09-12 21:30:00                       0.0   

                              bg max_peaks_above_mean  hour_of_day  hour_sin  \
id       datetime                                                              
41131654 2019-09-12 18:00:00                      0.0     0.782609  0.000000   
         2019-09-12 18:30:00                      0.0     0.782609  0.000000   
         2019-09-12 19:00:00                      1.0     0.826087  0.017037   
         2019-09-12 19:30:00                      0.0     0.826087  0.017037   
         2019-09-12 21:30:00                      0.0     0.913043  0.146447   

                              hour_cos  
id       datetime                       
41131654 2019-09-12 18:00:00  0.46